In [ ]:
import copy
import os
import random
import time

import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, models, transforms


# Seed 설정 함수
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# 데이터셋 분리 함수
def split_dataset(csv_path, train_save_path, valid_save_path, test_size, random_seed):
    df = pd.read_csv(csv_path, header=None, names=['path', 'label'])
    train_df, valid_df = train_test_split(df, test_size=test_size, random_state=random_seed, stratify=df['label'])
    train_df.to_csv(train_save_path, index=False)
    valid_df.to_csv(valid_save_path, index=False)

# 데이터셋 클래스
class KoreanHandwritingDataset(Dataset):
    def __init__(self, csv_file, image_dir, label_file, transform=None):
        self.dataset = pd.read_csv(csv_file, header=None, names=['path', 'label'])
        self.image_dir = image_dir
        self.label_file = label_file
        self.transform = transform
        with open(self.label_file, 'r', encoding='utf-8') as f:
            hangul_chars = [line.strip() for line in f.readlines()]
        self.label_mapping = {char: idx for idx, char in enumerate(hangul_chars)}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.dataset.iloc[idx]['path'])
        # 그레이스케일로 이미지 열기
        image = Image.open(img_path).convert("L")
        label = self.dataset.iloc[idx]['label']
        label = self.label_mapping[label]
        if self.transform:
            image = self.transform(image)
        return image, label

# 데이터 변환
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # 그레이스케일 이미지를 3채널로 복제
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 각 채널에 대한 평균 및 표준편차
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # 동일한 처리
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 데이터셋 및 데이터 로더 준비
# CSV 파일 분리
split_dataset('image-data/labels-map.csv', 'image-data/train-labels.csv', 'image-data/valid-labels.csv', 0.2, 42)

train_csv_file = "./image-data/train-labels.csv"
valid_csv_file = "./image-data/valid-labels.csv"
label_file = "./labels/256-common-hangul.txt"
image_dir = "./image-data/hangul-images"

train_dataset = KoreanHandwritingDataset(train_csv_file, image_dir, label_file, train_transform)
valid_dataset = KoreanHandwritingDataset(valid_csv_file, image_dir, label_file, valid_transform)

train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
valid_data_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=0)

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 준비
model_ft = models.efficientnet_b0(pretrained=True)
num_ftrs = model_ft.classifier[1].in_features
model_ft.classifier[1] = nn.Linear(num_ftrs, 256)
#model_ft = models.vgg16(pretrained=True)
#model_ft.classifier[6] = nn.Linear(in_features=4096, out_features=256)

model_ft = model_ft.to(device)

# 손실 함수 및 최적화 함수 설정
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# 훈련 함수
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in tqdm(range(num_epochs)):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        # 각 에포크는 학습 단계와 검증 단계를 갖습니다.
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # 모델을 학습 모드로 설정
            else:
                model.eval()   # 모델을 평가 모드로 설정
                
            running_loss = 0.0
            running_corrects = 0
            
            # 데이터를 배치 단위로 가져와 처리합니다.
            for inputs, labels in data_loader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # 파라미터 경사도를 0으로 설정
                optimizer.zero_grad()
                
                # 순전파
                # 학습 시에만 연산 기록을 추적
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # 학습 단계인 경우 역전파 + 최적화
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # 통계
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = running_loss / len(data_loader[phase].dataset)
            epoch_acc = running_corrects.double() / len(data_loader[phase].dataset)
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # 모델을 깊은 복사(deep copy)함
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # 가장 나은 모델 가중치를 불러옴
    model.load_state_dict(best_model_wts)
    return model

data_loader = {'train': train_data_loader, 'valid': valid_data_loader}

# 모델 훈련
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)


/Users/yeongseonchoe/GitHub/pytorch-hangul-recognition/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yeongseonchoe/GitHub/pytorch-hangul-recognition/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|                                                                                                                        | 0/1 [00:00<?, ?it/s]

Epoch 0/0
----------
